# Setup

In [1]:
# Import packages
import io
import os
import sys
import re
import json
import tqdm
import string
import tensorflow as tf

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

tf.__version__ # 2.x

'2.3.0'

In [2]:
# Globals
SEED = 21
NUM_NS = 4 # Negative sampling
WINDOW_SIZE = 2
BUFFER = 10000
BUFFER_BATCH_SIZE = 1024
VOCAB_SIZE = 1000
EMBEDDING_DIM = 128
SEQUENCE_LENGTH = 10

AUTOTUNE = tf.data.experimental.AUTOTUNE
PATH_TO_FILE = tf.keras.utils.get_file("shakespeare", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt")

# Skip-Gram Model

Word2Vec is not a singular algorithm, rather, it is a family of model architectures and optimizations that can be used to learn word embeddings from large datasets.

These papers proposed two methods for learning representations of words:

- Continuous Bag-of-Words Model which predicts the middle word based on surrounding context words. The context consists of a few words before and after the current (middle) word. This architecture is called a bag-of-words model as the order of words in the context is not important.

- Continuous Skip-gram Model which predict words within a certain range before and after the current word in the same sentence.

A skip-gram model predicts the context (or neighbors) of a word, given the word itself.
The model is trained on skip-grams, which are n-grams that allows tokens to be skipped.
The context of a word can be representend using a pair of (`target_words`, `context_word`) where `context_word` appears in the neighboring context of the `target_word`.

The `window_size` here determines the number of neighbors on either side of the `target_word` for context.

The objective of the skip-gram model is to maximize the probability of predicting context words given the `target_word`.

The computation of probability here would require to take softmax over the entire vocabulary which is often very large as any word from the vocabulary can be predicted for the given `target_word`. The `Noise Contrastive Estimation` loss function is an efficient approximation for a full softmax. Rather than taking the whole vocabulary as sample for probability distribution, negative sampling is used.

The simplified negative sampling objective for a `target_word` is to distinguish the context words from number of n negative samples drawn from a noise distribution P(W) of words. That is, an approximation of softmax over the entire vocabulary is to pose the loss for a `target_word` as a classification problem between context word and number of n negative samples.

A negative sample is defined as (`target_word`, `context_word`) pair such that the context word does not appear in the `window_size` of the `target_word`.

# Setup input pipeline

In [3]:
with open(PATH_TO_FILE) as f: 
    lines = f.read().splitlines()
    for line in lines[:20]:
        print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [4]:
# Create dataset
text_ds = tf.data.TextLineDataset(PATH_TO_FILE).filter(lambda x: tf.cast(tf.strings.length(x), bool))

for text in text_ds.take(1):
    print("Input text:", text)

Input text: tf.Tensor(b'First Citizen:', shape=(), dtype=string)


# Data processing

In [5]:
# # Standardize sentences
# def standardize_sequences(input_data):
#     input_data = tf.strings.lower(input_data)
#     return tf.strings.regex_replace(input_data, pattern=f"[{re.escape(string.punctuation)}]", rewrite="")

# # Test
# standardize_sequences("Hello, I'm nityan!")

In [6]:
# Define a vectorization layer
vectorization_layer = TextVectorization(
    standardize="lower_and_strip_punctuation", # Using default standardization
    max_tokens=VOCAB_SIZE,
    output_sequence_length=SEQUENCE_LENGTH,
    output_mode="int"
)

# Learn vectorizer
vectorization_layer.adapt(text_ds.batch(132))

In [7]:
# Vocabulary
vocabulary = vectorization_layer.get_vocabulary()
print(vocabulary[:5])

['', '[UNK]', 'the', 'and', 'to']


In [8]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorization_layer).unbatch()

In [9]:
# Get sequences out
sequences = list(text_vector_ds.as_numpy_iterator())

print("Number of sequences:", len(sequences))
print("First sequence:", sequences[0])

Number of sequences: 32777
First sequence: [ 89 270   0   0   0   0   0   0   0   0]


In [10]:
# Subsampling table
subsampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=VOCAB_SIZE)

In [11]:
targets, contexts, labels = list(), list(), list()

for sequence in tqdm.tqdm(sequences):
    # Get positive ngrams
    positive_ngrams, _ = tf.keras.preprocessing.sequence.skipgrams(
        sequence=sequence,
        vocabulary_size=VOCAB_SIZE,
        sampling_table=subsampling_table,
        window_size=WINDOW_SIZE,
        negative_samples=0
    )

    for target_w, context_w in positive_ngrams:
        # Create context tensor
        context_class = tf.expand_dims(tf.constant([context_w], dtype="int64"), 1)

        # Get negative contexts
        negative_context_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
            true_classes=context_class,
            num_true=1,
            num_sampled=NUM_NS,
            unique=True,
            range_max=VOCAB_SIZE,
            seed=SEED
        )
        negative_context_candidates = tf.expand_dims(negative_context_candidates, 1)

        # Add negative and positive contexts together
        context = tf.concat([context_class, negative_context_candidates], 0)

        # Create label
        label = tf.constant([1] + [0]*NUM_NS, dtype="int64")

        # Store
        targets.append(target_w)
        contexts.append(context)
        labels.append(label)

100%|██████████| 32777/32777 [00:09<00:00, 3613.76it/s]


In [12]:
print(len(targets), len(contexts), len(labels))

35215 35215 35215


In [13]:
# Create optimized model input
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER).batch(BUFFER_BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)

# Modelling

The Word2Vec model can be implemented as a classifier to distinguish between true context words from skip-grams and false context words obtained through negative sampling.

You can perform a dot product between the embeddings of target and context words to obtain predictions for labels and compute loss against true labels in the dataset.

In [14]:
class Word2Vec(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, num_ns):
        super(Word2Vec, self).__init__()
        self.target_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=1, name="w2v_embedding")
        self.context_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=num_ns+1)
        self.dots = tf.keras.layers.Dot(axes=(3,2))
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, pair):
        target, context = pair
        we = self.target_embedding(target)
        ce = self.context_embedding(context)
        dots = self.dots([ce, we])
        return self.flatten(dots)

In [15]:
# Compile model
word2vec = Word2Vec(VOCAB_SIZE, EMBEDDING_DIM, NUM_NS)

word2vec.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

In [16]:
# Log modelling with tensorboard for analysis and visualization
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
history = word2vec.fit(dataset, epochs=50, callbacks=[tensorboard_callback])

Epoch 1/50
 1/34 [..............................] - ETA: 0s - loss: 1.6092 - accuracy: 0.1982WARNING:tensorflow:From /home/nityan/anaconda3/envs/dl-env/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
34/34 [==============================] - 0s 4ms/step - loss: 1.6071 - accuracy: 0.2673
Epoch 2/50
34/34 [==============================] - 0s 3ms/step - loss: 1.5883 - accuracy: 0.4857
Epoch 3/50
34/34 [==============================] - 0s 3ms/step - loss: 1.5477 - accuracy: 0.5190
Epoch 4/50
34/34 [==============================] - 0s 3ms/step - loss: 1.4909 - accuracy: 0.4916
Epoch 5/50
34/34 [==============================] - 0s 3ms/step - loss: 1.4403 - accuracy: 0.4771
Epoch 6/50
34/34 [==============================] - 0s 3ms/step - loss: 1.3975 - accuracy: 0.4846
Epoch 7/50
34/34 [==========

In [17]:
# Tensorboard for embedding visualization
# ! tensorboard --logdir logs

# Embedding Lookup

In [18]:
# Get weights from layer
weights = word2vec.get_layer("w2v_embedding").get_weights()[0]
vocab = vectorization_layer.get_vocabulary()

In [19]:
# Create and save metadata files
out_v = io.open("/tmp/vectors.tsv", "w", encoding="utf-8")
out_m = io.open("/tmp/metadata.tsv", "w", encoding="utf-8")

for index, word in enumerate(vocab):
    if  index == 0: continue # skip 0, it"s padding.
    vec = weights[index] 
    out_v.write("\t".join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")
out_v.close()
out_m.close()